## Peer-graded Assignment: 
## Segmenting and Clustering Neighborhoods in Toronto
### Importing Libraries 

In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto based on the postalcode and borough information.. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas  dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

In [1]:
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests 
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library
from bs4 import BeautifulSoup as bs
import lxml
print('Libraries imported.')

Libraries imported.


## PART 1 : Scraping Toronto Neighbourhood from Wikipedia

For Toronto data neighbourhood data, i scrape Wikipedia file https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [2]:
r = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = bs(r.text, 'html.parser')
table=soup.find('table')
len(table)

2

In [3]:
data = table.find_all('td')
len(data)

180

In [4]:
neighbourhood = [] #init list
for i,item in enumerate(data) :
    neigh = [] #init list
    span_ = item.span.text.strip()
    if not(span_ == "Not assigned"):
        text_ = item.p.text.strip()
        poscode = text_[0:3]
        try :
            borough = item.find('a').text.strip()
        except :
            borough = "NA"
        try :
            neigh = text_.split('(')[1] #remove '('
            neigh = neigh[:-1]  #remove ')' 
            #split neighbourhood by '/' and convert to list
            #neigh = neigh.split('/')
            neigh = (((neigh.replace(' /',',')).replace(')',' ')).strip(' '))
        except:
            neigh = "NA"
        #print(f"<{poscode}><{borough}><{neigh}>")
        neigh2 = {
            "PostalCode" : poscode,
            "Borough" : borough,
            "Neighborhood" :neigh,
            }
        neighbourhood.append(neigh2)
        #end if
    #end for i
df_neigh = pd.DataFrame(neighbourhood)


In [5]:
df_neigh['Borough']=df_neigh['Borough'].replace({'Business reply mail':'East Toronto',
                                                 'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                                 'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                                 'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                                 'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [6]:
df_neigh.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [7]:
df_neigh.tail(5)

,PostalCode,Borough,Neighborhood
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


In [8]:
df_neigh.shape

(103, 3)

#### we have 103 rows of Toronto Neighbourhood from the WIkipedia datasheet

## PART 2 : 

1. Reading Lat Long file from Geospatial datasheet at
https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv

In [9]:
df_latlong = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv")

In [10]:
df_latlong.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
df_latlong.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

In [12]:
df_neigh.head(5)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [13]:
# merge Lat Long and create Toronto Neighbourhood dataframe
Toronto_data = df_neigh.join(df_latlong.set_index('PostalCode'), on='PostalCode')

In [14]:
Toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [15]:
Toronto_data.tail()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999


#### new Toronto_data having information of neighborhoood and its latitude and longitude above

### PART 3:
#### Explore and cluster the neighborhoods in Toronto. You can decide to work with only boroughs that contain the word Toronto and then replicate the same analysis we did to the New York City data. 


1. to add enough Markdown cells to explain what you decided to do and to report any observations you make. 
2. to generate maps to visualize your neighborhoods and how they cluster together. 

Get the coordinate for Toronto for opening map later

In [16]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


### 1. Visualize Toronto Neighborhood

In [17]:
#Toronto_data = df_Toronto
# create map of Toronto using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

### Work with Foursquare to explore the Toronto area
### Define Foursquare Credentials and Version

Randomnly pick an area in TORONTO

In [19]:
area_no = 74
neighborhood_name = Toronto_data.loc[area_no,'Neighborhood']
neighborhood_latitude = Toronto_data.loc[area_no,'Latitude']
neighborhood_longitude = Toronto_data.loc[area_no,'Longitude']
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Annex, North Midtown, Yorkville are 43.6727097, -79.4056784.


In [20]:
# type your answer here
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '60ff83d41d562a191a280352'},
 'response': {'headerLocation': 'The Annex',
  'headerFullLocation': 'The Annex, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 21,
  'suggestedBounds': {'ne': {'lat': 43.6772097045, 'lng': -79.39946849197052},
   'sw': {'lat': 43.668209695499996, 'lng': -79.41188830802948}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ae7bc29f964a52084ad21e3',
       'name': "Ezra's Pound",
       'location': {'address': '238 Dupont St',
        'crossStreet': 'Spadina Ave',
        'lat': 43.67515283323029,
        'lng': -79.40585846415303,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.67515283323029,
          'lng': -79.40585846415303}],
        'distance': 272,
  

In [22]:
'There are {} popular spots around {}.'.format(len(results['response']['groups'][0]['items']),neighborhood_name)

'There are 21 popular spots around The Annex, North Midtown, Yorkville.'

### 2. Function to get popular venue in the neighbourhood

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    #will put all venue in venue_list into dataframe
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

create Toronto venus dataframe

In [24]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills Nort
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills Sout
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth W

In [25]:

Toronto_venues.tail()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2148,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Jim & Maria's No Frills,43.631152,-79.518617,Grocery Store
2149,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Islington Florist & Nursery,43.630156,-79.518718,Flower Shop
2150,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Koala Tan Tanning Salon & Sunless Spa,43.631370,-79.519006,Tanning Salon
2151,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Value Village,43.631269,-79.518238,Thrift / Vintage Store
2152,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,Kingsway Boxing Club,43.627254,-79.526684,Gym


Let's check how many venues were returned for each neighborhood

In [26]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",24,24,24,24,24,24
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
Berczy Park,56,56,56,56,56,56
"Birch Cliff, Cliffside West",5,5,5,5,5,5
"Brockton, Parkdale Village, Exhibition Place",24,24,24,24,24,24
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15


Let's find out how many unique categories can be curated from all the returned venues

In [27]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 277 uniques categories.


### 3. Analyze Each Neighborhood

In [28]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']])

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighborhood,Venue Category_Accessories Store,Venue Category_Adult Boutique,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,Venue Category_Airport Lounge,Venue Category_Airport Service,Venue Category_Airport Terminal,Venue Category_American Restaurant,Venue Category_Antique Shop,Venue Category_Aquarium,Venue Category_Art Gallery,Venue Category_Art Museum,Venue Category_Arts & Crafts Store,Venue Category_Asian Restaurant,Venue Category_Athletics & Sports,Venue Category_Auto Garage,Venue Category_Auto Workshop,Venue Category_BBQ Joint,Venue Category_Baby Store,Venue Category_Bagel Shop,Venue Category_Bakery,Venue Category_Bank,Venue Category_Bar,Venue Category_Baseball Field,Venue Category_Baseball Stadium,Venue Category_Basketball Court,Venue Category_Basketball Stadium,Venue Category_Beach,Venue Category_Bed & Breakfast,Venue Category_Beer Bar,Venue Category_Beer Store,Venue Category_Belgian Restaurant,Venue Category_Bike Shop,Venue Category_Bistro,Venue Category_Board Shop,Venue Category_Boat or Ferry,Venue Category_Bookstore,Venue Category_Boutique,Venue Category_Brazilian Restaurant,Venue Category_Breakfast Spot,Venue Category_Brewery,Venue Category_Bridal Shop,Venue Category_Bubble Tea Shop,Venue Category_Building,Venue Category_Burger Joint,Venue Category_Burrito Place,Venue Category_Bus Line,Venue Category_Bus Station,Venue Category_Business Service,Venue Category_Butcher,Venue Category_Café,Venue Category_Cajun / Creole Restaurant,Venue Category_Camera Store,Venue Category_Candy Store,Venue Category_Caribbean Restaurant,Venue Category_Cheese Shop,Venue Category_Chinese Restaurant,Venue Category_Chocolate Shop,Venue Category_Church,Venue Category_Climbing Gym,Venue Category_Clothing Store,Venue Category_Cocktail Bar,Venue Category_Coffee Shop,Venue Category_College Arts Building,Venue Category_College Auditorium,Venue Category_College Cafeteria,Venue Category_College Gym,Venue Category_College Rec Center,Venue Category_College Stadium,Venue Category_Colombian Restaurant,Venue Category_Comfort Food Restaurant,Venue Category_Comic Shop,Venue Category_Concert Hall,Venue Category_Construction & Landscaping,Venue Category_Convenience Store,Venue Category_Cosmetics Shop,Venue Category_Coworking Space,Venue Category_Creperie,Venue Category_Cuban Restaurant,Venue Category_Cupcake Shop,Venue Category_Curling Ice,Venue Category_Dance Studio,Venue Category_Deli / Bodega,Venue Category_Department Store,Venue Category_Dessert Shop,Venue Category_Dim Sum Restaurant,Venue Category_Diner,Venue Category_Discount Store,Venue Category_Distribution Center,Venue Category_Dog Run,Venue Category_Doner Restaurant,Venue Category_Donut Shop,Venue Category_Drugstore,Venue Category_Dumpling Restaurant,Venue Category_Eastern European Restaurant,Venue Category_Electronics Store,Venue Category_Escape Room,Venue Category_Ethiopian Restaurant,Venue Category_Event Space,Venue Category_Falafel Restaurant,Venue Category_Farm,Venue Category_Farmers Market,Venue Category_Fast Food Restaurant,Venue Category_Field,Venue Category_Filipino Restaurant,Venue Category_Fish & Chips Shop,Venue Category_Fish Market,Venue Category_Flea Market,Venue Category_Flower Shop,Venue Category_Food & Drink Shop,Venue Category_Food Court,Venue Category_Food Truck,Venue Category_Fountain,Venue Category_French Restaurant,Venue Category_Fried Chicken Joint,Venue Category_Frozen Yogurt Shop,Venue Category_Fruit & Vegetable Store,Venue Category_Furniture / Home Store,Venue Category_Gaming Cafe,Venue Category_Garden,Venue Category_Garden Center,Venue Category_Gas Station,Venue Category_Gastropub,Venue Category_Gay Bar,Venue Category_General Entertainment,Venue Category_General Travel,Venue Category_German Restaurant,Venue Category_Gift Shop,Venue Category_Gluten-free Restaurant,Venue Category_Golf Course,Venue Category_Gourmet Shop,Venue Category_Greek Restaurant,Venue Category_Grocery Store,Venue Category_Gym,Venue Category_Gym / Fitness Center,Ve

And let's examine the new dataframe size.

In [29]:
Toronto_onehot.shape

(2153, 278)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [30]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped

,Neighborhood,Venue Category_Accessories Store,Venue Category_Adult Boutique,Venue Category_Airport,Venue Category_Airport Food Court,Venue Category_Airport Gate,Venue Category_Airport Lounge,Venue Category_Airport Service,Venue Category_Airport Terminal,Venue Category_American Restaurant,Venue Category_Antique Shop,Venue Category_Aquarium,Venue Category_Art Gallery,Venue Category_Art Museum,Venue Category_Arts & Crafts Store,Venue Category_Asian Restaurant,Venue Category_Athletics & Sports,Venue Category_Auto Garage,Venue Category_Auto Workshop,Venue Category_BBQ Joint,Venue Category_Baby Store,Venue Category_Bagel Shop,Venue Category_Bakery,Venue Category_Bank,Venue Category_Bar,Venue Category_Baseball Field,Venue Category_Baseball Stadium,Venue Category_Basketball Court,Venue Category_Basketball Stadium,Venue Category_Beach,Venue Category_Bed & Breakfast,Venue Category_Beer Bar,Venue Category_Beer Store,Venue Category_Belgian Restaurant,Venue Category_Bike Shop,Venue Category_Bistro,Venue Category_Board Shop,Venue Category_Boat or Ferry,Venue Category_Bookstore,Venue Category_Boutique,Venue Category_Brazilian Restaurant,Venue Category_Breakfast Spot,Venue Category_Brewery,Venue Category_Bridal Shop,Venue Category_Bubble Tea Shop,Venue Category_Building,Venue Category_Burger Joint,Venue Category_Burrito Place,Venue Category_Bus Line,Venue Category_Bus Station,Venue Category_Business Service,Venue Category_Butcher,Venue Category_Café,Venue Category_Cajun / Creole Restaurant,Venue Category_Camera Store,Venue Category_Candy Store,Venue Category_Caribbean Restaurant,Venue Category_Cheese Shop,Venue Category_Chinese Restaurant,Venue Category_Chocolate Shop,Venue Category_Church,Venue Category_Climbing Gym,Venue Category_Clothing Store,Venue Category_Cocktail Bar,Venue Category_Coffee Shop,Venue Category_College Arts Building,Venue Category_College Auditorium,Venue Category_College Cafeteria,Venue Category_College Gym,Venue Category_College Rec Center,Venue Category_College Stadium,Venue Category_Colombian Restaurant,Venue Category_Comfort Food Restaurant,Venue Category_Comic Shop,Venue Category_Concert Hall,Venue Category_Construction & Landscaping,Venue Category_Convenience Store,Venue Category_Cosmetics Shop,Venue Category_Coworking Space,Venue Category_Creperie,Venue Category_Cuban Restaurant,Venue Category_Cupcake Shop,Venue Category_Curling Ice,Venue Category_Dance Studio,Venue Category_Deli / Bodega,Venue Category_Department Store,Venue Category_Dessert Shop,Venue Category_Dim Sum Restaurant,Venue Category_Diner,Venue Category_Discount Store,Venue Category_Distribution Center,Venue Category_Dog Run,Venue Category_Doner Restaurant,Venue Category_Donut Shop,Venue Category_Drugstore,Venue Category_Dumpling Restaurant,Venue Category_Eastern European Restaurant,Venue Category_Electronics Store,Venue Category_Escape Room,Venue Category_Ethiopian Restaurant,Venue Category_Event Space,Venue Category_Falafel Restaurant,Venue Category_Farm,Venue Category_Farmers Market,Venue Category_Fast Food Restaurant,Venue Category_Field,Venue Category_Filipino Restaurant,Venue Category_Fish & Chips Shop,Venue Category_Fish Market,Venue Category_Flea Market,Venue Category_Flower Shop,Venue Category_Food & Drink Shop,Venue Category_Food Court,Venue Category_Food Truck,Venue Category_Fountain,Venue Category_French Restaurant,Venue Category_Fried Chicken Joint,Venue Category_Frozen Yogurt Shop,Venue Category_Fruit & Vegetable Store,Venue Category_Furniture / Home Store,Venue Category_Gaming Cafe,Venue Category_Garden,Venue Category_Garden Center,Venue Category_Gas Station,Venue Category_Gastropub,Venue Category_Gay Bar,Venue Category_General Entertainment,Venue Category_General Travel,Venue Category_German Restaurant,Venue Category_Gift Shop,Venue Category_Gluten-free Restaurant,Venue Category_Golf Course,Venue Category_Gourmet Shop,Venue Category_Greek Restaurant,Venue Category_Grocery Store,Venue Category_Gym,Venue Category_Gym / Fitness Center,Ve

In [31]:
Toronto_grouped.shape

(100, 278)

#### Let's print each neighborhood along with the top 5 most common venues

In [32]:
num_top_venues = 5

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                                      venue  freq
0  Venue Category_Latin American Restaurant  0.25
1               Venue Category_Skating Rink  0.25
2             Venue Category_Breakfast Spot  0.25
3                     Venue Category_Lounge  0.25
4        Venue Category_Moroccan Restaurant  0.00


----Alderwood, Long Branch----
                           venue  freq
0     Venue Category_Pizza Place  0.25
1     Venue Category_Coffee Shop  0.12
2    Venue Category_Skating Rink  0.12
3  Venue Category_Sandwich Place  0.12
4             Venue Category_Pub  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
                                      venue  freq
0                       Venue Category_Bank  0.08
1                Venue Category_Coffee Shop  0.08
2                Venue Category_Gas Station  0.04
3                  Venue Category_Gift Shop  0.04
4  Venue Category_Middle Eastern Restaurant  0.04


----Bayview Village----
                               

4  Venue Category_Accessories Store  0.00


----Garden District, Ryerson----
                            venue  freq
0      Venue Category_Coffee Shop  0.11
1   Venue Category_Clothing Store  0.09
2            Venue Category_Hotel  0.03
3  Venue Category_Bubble Tea Shop  0.03
4   Venue Category_Cosmetics Shop  0.03


----Glencairn----
                                venue  freq
0  Venue Category_Japanese Restaurant   0.2
1               Venue Category_Bakery   0.2
2        Venue Category_Metro Station   0.2
3     Venue Category_Asian Restaurant   0.2
4                 Venue Category_Park   0.2


----Golden Mile, Clairlea, Oakridge----
                         venue  freq
0      Venue Category_Bus Line   0.2
1        Venue Category_Bakery   0.2
2  Venue Category_Soccer Field   0.1
3          Venue Category_Park   0.1
4  Venue Category_Intersection   0.1


----Guildwood, Morningside, West Hill----
                               venue  freq
0      Venue Category_Medical Center  0.12
1    

                              venue  freq
0               Venue Category_Café  0.14
1     Venue Category_Sandwich Place  0.14
2        Venue Category_Coffee Shop  0.14
3           Venue Category_Pharmacy  0.05
4  Venue Category_Indian Restaurant  0.05


----The Beaches----
                                  venue  freq
0      Venue Category_Health Food Store  0.25
1                    Venue Category_Pub  0.25
2                  Venue Category_Trail  0.25
3           Venue Category_Neighborhood  0.25
4  Venue Category_Performing Arts Venue  0.00


----The Danforth  East----
                              venue  freq
0  Venue Category_Convenience Store  0.33
1      Venue Category_Metro Station  0.33
2               Venue Category_Park  0.33
3  Venue Category_Accessories Store  0.00
4  Venue Category_Mobile Phone Shop  0.00


----The Danforth West, Riverdale----
                                   venue  freq
0        Venue Category_Greek Restaurant  0.17
1      Venue Category_Italian Restau

Let's put that into a pandas dataframe

In [33]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [34]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Venue Category_Latin American Restaurant,Venue Category_Skating Rink,Venue Category_Breakfast Spot,Venue Category_Lounge,Venue Category_Moroccan Restaurant,Venue Category_Monument / Landmark,Venue Category_Molecular Gastronomy Restaurant,Venue Category_Modern European Restaurant,Venue Category_Mobile Phone Shop,Venue Category_Miscellaneous Shop
1,"Alderwood, Long Branch",Venue Category_Pizza Place,Venue Category_Coffee Shop,Venue Category_Skating Rink,Venue Category_Sandwich Place,Venue Category_Pub,Venue Category_Pharmacy,Venue Category_Gym,Venue Category_Peking Duck Restaurant,Venue Category_Persian Restaurant,Venue Category_Mediterranean Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Venue Category_Bank,Venue Category_Coffee Shop,Venue Category_Gas Station,Venue Category_Gift Shop,Venue Category_Middle Eastern Restaurant,Venue Category_Shopping Mall,Venue Category_Mobile Phone Shop,Venue Category_Sandwich Place,Venue Category_Fried Chicken Joint,Venue Category_Frozen Yogurt Shop
3,Bayview Village,Venue Category_Japanese Restaurant,Venue Category_Café,Venue Category_Bank,Venue Category_Chinese Restaurant,Venue Category_Accessories Store,Venue Category_Movie Theater,Venue Category_Motel,Venue Category_Moroccan Restaurant,Venue Category_Monument / Landmark,Venue Category_Molecular Gastronomy Restaurant
4,"Bedford Park, Lawrence Manor East",Venue Category_Italian Restaurant,Venue Category_Coffee Shop,Venue Category_Sandwich Place,Venue Category_Comfort Food Restaurant,Venue Category_Pizza Place,Venue Category_Cupcake Shop,Venue Category_Pub,Venue Category_Restaurant,Venue Category_Café,Venue Category_Butcher


### 4. Cluster Neighborhoods
#### Run k-means to cluster the neighborhood into 5 clusters.

In [36]:
neighborhoods_venues_sorted['1st Most Common Venue'].unique()

array(['Venue Category_Latin American Restaurant',
       'Venue Category_Pizza Place', 'Venue Category_Bank',
       'Venue Category_Japanese Restaurant',
       'Venue Category_Italian Restaurant', 'Venue Category_Bakery',
       'Venue Category_College Stadium', 'Venue Category_Café',
       'Venue Category_Airport Terminal', "Venue Category_Women's Store",
       'Venue Category_Thai Restaurant', 'Venue Category_Coffee Shop',
       'Venue Category_Grocery Store', 'Venue Category_Bar',
       'Venue Category_Motel', 'Venue Category_Sandwich Place',
       'Venue Category_Hotel', 'Venue Category_Discount Store',
       'Venue Category_Caribbean Restaurant',
       'Venue Category_Clothing Store',
       'Venue Category_Indian Restaurant', 'Venue Category_Food Truck',
       'Venue Category_Airport', 'Venue Category_Athletics & Sports',
       'Venue Category_Pharmacy', 'Venue Category_Light Rail Station',
       'Venue Category_Jewelry Store', 'Venue Category_Bus Line',
       'Venu

In [37]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 3])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [38]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data

# merge Toronto_grouped with Toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.tail() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,0.0,Venue Category_Smoke Shop,Venue Category_Pool,Venue Category_Park,Venue Category_River,Venue Category_Miscellaneous Shop,Venue Category_Moroccan Restaurant,Venue Category_Monument / Landmark,Venue Category_Molecular Gastronomy Restaurant,Venue Category_Modern European Restaurant,Venue Category_Mobile Phone Shop
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,0.0,Venue Category_Coffee Shop,Venue Category_Sushi Restaurant,Venue Category_Japanese Restaurant,Venue Category_Gay Bar,Venue Category_Restaurant,Venue Category_Fast Food Restaurant,Venue Category_Pub,Venue Category_Men's Store,Venue Category_Mediterranean Restaurant,Venue Category_Yoga Studio
100,M7Y,East Toronto,Enclave of M4L,43.662744,-79.321558,0.0,Venue Category_Light Rail Station,Venue Category_Pizza Place,Venue Category_Burrito Place,Venue Category_Brewery,Venue Category_Auto Workshop,Venue Category_Restaurant,Venue Category_Garden Center,Venue Category_Skate Park,Venue Category_Garden,Venue Category_Gym / Fitness Center
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,2.0,Venue Category_Baseball Field,Venue Category_Accessories Store,Venue Category_Mobile Phone Shop,Venue Category_Motel,Venue Category_Moroccan Restaurant,Venue Category_Monument / Landmark,Venue Category_Molecular Gastronomy Restaurant,Venue Category_Modern European Restaurant,Venue Category_Miscellaneous Shop,Venue Category_Museum
102,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",43.628841,-79.520999,0.0,Venue Category_Hardware Store,Venue Category_Thrift / Vintage Store,Venue Category_Fast Food Restaurant,Venue Category_Wings Joint,Venue Category_Flower Shop,Venue Category_Convenience Store,Venue Category_Burger Joint,Venue Category_Tanning Salon,Venue Category_Bakery,Venue Category_Gym


Finally, let's visualize the resulting clusters

In [39]:
Toronto_merged['Cluster Labels'].unique()

array([ 3.,  0., nan,  4.,  1.,  2.])

#simply drop whole row with NaN in "Most Common Venue" column

In [40]:
# simply drop whole row with NaN in "Most Common Venue" column
Toronto_merged.dropna(subset=["1st Most Common Venue"], axis=0, inplace=True)

# reset index, because we droped two rows
Toronto_merged.reset_index(drop=True, inplace=True)

In [41]:
Toronto_merged['Cluster Labels'].unique()

array([3., 0., 4., 1., 2.])

In [43]:
#change the Cluster Labels to int type for map visualization
Toronto_merged['Cluster Labels'] = Toronto_merged['Cluster Labels'].astype(int)

In [44]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Cluster 1

In [45]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,0,Venue Category_Pizza Place,Venue Category_Intersection,Venue Category_Hockey Arena,Venue Category_Portuguese Restaurant,Venue Category_Coffee Shop,Venue Category_Modern European Restaurant,Venue Category_Movie Theater,Venue Category_Motel,Venue Category_Moroccan Restaurant,Venue Category_Monument / Landmark
2,Downtown Toronto,0,Venue Category_Coffee Shop,Venue Category_Bakery,Venue Category_Park,Venue Category_Pub,Venue Category_Breakfast Spot,Venue Category_Theater,Venue Category_Café,Venue Category_Shoe Store,Venue Category_Beer Store,Venue Category_Sushi Restaurant
3,North York,0,Venue Category_Clothing Store,Venue Category_Accessories Store,Venue Category_Vietnamese Restaurant,Venue Category_Sporting Goods Shop,Venue Category_Event Space,Venue Category_Furniture / Home Store,Venue Category_Coffee Shop,Venue Category_Boutique,Venue Category_Women's Store,Venue Category_Other Great Outdoors
4,Queen's Park,0,Venue Category_Coffee Shop,Venue Category_Sushi Restaurant,Venue Category_Café,Venue Category_Sandwich Place,Venue Category_Beer Bar,Venue Category_Fried Chicken Joint,Venue Category_Smoothie Shop,Venue Category_Mexican Restaurant,Venue Category_Burrito Place,Venue Category_Yoga Studio
5,Scarborough,0,Venue Category_Fast Food Restaurant,Venue Category_Print Shop,Venue Category_Miscellaneous Shop,Venue Category_Motel,Venue Category_Moroccan Restaurant,Venue Category_Monument / Landmark,Venue Category_Molecular Gastronomy Restaurant,Venue Category_Modern European Restaurant,Venue Category_Mobile Phone Shop,Venue Category_Middle Eastern Restaurant
6,North York,0,Venue Category_Caribbean Restaurant,Venue Category_Gym,Venue Category_Café,Venue Category_Baseball Field,Venue Category_Dessert Shop,Venue Category_Japanese Restaurant,Venue Category_Mobile Phone Shop,Venue Category_Miscellaneous Shop,Venue Category_Modern European Restaurant,Venue Category_Molecular Gastronomy Restaurant
7,East York,0,Venue Category_Pizza Place,Venue Category_Athletics & Sports,Venue Category_Pharmacy,Venue Category_Pet Store,Venue Category_Café,Venue Category_Breakfast Spot,Venue Category_Bank,Venue Category_Gastropub,Venue Category_Intersection,Venue Category_Flea Market
8,Downtown Toronto,0,Venue Category_Coffee Shop,Venue Category_Clothing Store,Venue Category_Hotel,Venue Category_Bubble Tea Shop,Venue Category_Cosmetics Shop,Venue Category_Café,Venue Category_Japanese Restaurant,Venue Category_Theater,Venue Category_Diner,Venue Category_Fast Food Restaurant
9,North York,0,Venue Category_Japanese Restaurant,Venue Category_Bakery,Venue Category_Metro Station,Venue Category_Asian Restaurant,Venue Category_Park,Venue Category_Molecular Gastronomy Restaurant,Venue Category_Movie Theater,Venue Category_Motel,Venue Category_Moroccan Restaurant,Venue Category_Monument / Landmark
10,Etobicoke,0,Venue Category_Filipino Restaurant,Venue Category_Bakery,Venue Category_Mobile Phone Shop,Venue Category_Motel,Venue Category_Moroccan Restaurant,Venue Category_Monument / Landmark,Venue Category_Molecular Gastronomy Restaurant,Venue Category_Modern European Restaurant,Venue Category_Miscellaneous Shop,Venue Category_Museum


#### Cluster 2

In [46]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
31,Scarborough,1,Venue Category_Playground,Venue Category_Mobile Phone Shop,Venue Category_Movie Theater,Venue Category_Motel,Venue Category_Moroccan Restaurant,Venue Category_Monument / Landmark,Venue Category_Molecular Gastronomy Restaurant,Venue Category_Modern European Restaurant,Venue Category_Miscellaneous Shop,Venue Category_Martial Arts School


#### Cluster 3

In [47]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
55,North York,2,Venue Category_Furniture / Home Store,Venue Category_Baseball Field,Venue Category_Accessories Store,Venue Category_Martial Arts School,Venue Category_Motel,Venue Category_Moroccan Restaurant,Venue Category_Monument / Landmark,Venue Category_Molecular Gastronomy Restaurant,Venue Category_Modern European Restaurant,Venue Category_Mobile Phone Shop
98,Etobicoke,2,Venue Category_Baseball Field,Venue Category_Accessories Store,Venue Category_Mobile Phone Shop,Venue Category_Motel,Venue Category_Moroccan Restaurant,Venue Category_Monument / Landmark,Venue Category_Molecular Gastronomy Restaurant,Venue Category_Modern European Restaurant,Venue Category_Miscellaneous Shop,Venue Category_Museum


#### Cluster 4

In [48]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,3,Venue Category_Fast Food Restaurant,Venue Category_Park,Venue Category_Food & Drink Shop,Venue Category_Middle Eastern Restaurant,Venue Category_Moroccan Restaurant,Venue Category_Monument / Landmark,Venue Category_Molecular Gastronomy Restaurant,Venue Category_Modern European Restaurant,Venue Category_Mobile Phone Shop,Venue Category_Miscellaneous Shop
20,York,3,Venue Category_Women's Store,Venue Category_Pool,Venue Category_Park,Venue Category_Accessories Store,Venue Category_Miscellaneous Shop,Venue Category_Motel,Venue Category_Moroccan Restaurant,Venue Category_Monument / Landmark,Venue Category_Molecular Gastronomy Restaurant,Venue Category_Modern European Restaurant
34,East York,3,Venue Category_Convenience Store,Venue Category_Metro Station,Venue Category_Park,Venue Category_Accessories Store,Venue Category_Mobile Phone Shop,Venue Category_Motel,Venue Category_Moroccan Restaurant,Venue Category_Monument / Landmark,Venue Category_Molecular Gastronomy Restaurant,Venue Category_Modern European Restaurant
39,North York,3,Venue Category_Airport,Venue Category_Business Service,Venue Category_Park,Venue Category_Accessories Store,Venue Category_Mobile Phone Shop,Venue Category_Motel,Venue Category_Moroccan Restaurant,Venue Category_Monument / Landmark,Venue Category_Molecular Gastronomy Restaurant,Venue Category_Modern European Restaurant
50,North York,3,Venue Category_Park,Venue Category_Accessories Store,Venue Category_Miscellaneous Shop,Venue Category_Motel,Venue Category_Moroccan Restaurant,Venue Category_Monument / Landmark,Venue Category_Molecular Gastronomy Restaurant,Venue Category_Modern European Restaurant,Venue Category_Mobile Phone Shop,Venue Category_Middle Eastern Restaurant
59,Lawrence Park,3,Venue Category_Bus Line,Venue Category_Park,Venue Category_Swim School,Venue Category_Accessories Store,Venue Category_Mobile Phone Shop,Venue Category_Moroccan Restaurant,Venue Category_Monument / Landmark,Venue Category_Molecular Gastronomy Restaurant,Venue Category_Modern European Restaurant,Venue Category_Miscellaneous Shop
62,York,3,Venue Category_Convenience Store,Venue Category_Park,Venue Category_Accessories Store,Venue Category_Miscellaneous Shop,Venue Category_Motel,Venue Category_Moroccan Restaurant,Venue Category_Monument / Landmark,Venue Category_Molecular Gastronomy Restaurant,Venue Category_Modern European Restaurant,Venue Category_Mobile Phone Shop
64,North York,3,Venue Category_Convenience Store,Venue Category_Park,Venue Category_Accessories Store,Venue Category_Miscellaneous Shop,Venue Category_Motel,Venue Category_Moroccan Restaurant,Venue Category_Monument / Landmark,Venue Category_Molecular Gastronomy Restaurant,Venue Category_Modern European Restaurant,Venue Category_Mobile Phone Shop
81,Moore Park,3,Venue Category_Park,Venue Category_Summer Camp,Venue Category_Accessories Store,Venue Category_Middle Eastern Restaurant,Venue Category_Moroccan Restaurant,Venue Category_Monument / Landmark,Venue Category_Molecular Gastronomy Restaurant,Venue Category_Modern European Restaurant,Venue Category_Mobile Phone Shop,Venue Category_Miscellaneous Shop
89,Downtown Toronto,3,Venue Category_Park,Venue Category_Playground,Venue Category_Trail,Venue Category_Accessories Store,Venue Category_Mobile Phone Shop,Venue Category_Motel,Venue Category_Moroccan Restaurant,Venue Category_Monument / Landmark,Venue Category_Molecular Gastronomy Restaurant,Venue Category_Modern European Restaurant


#### Cluster 5

In [49]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Scarborough,4,Venue Category_Bar,Venue Category_Accessories Store,Venue Category_Mobile Phone Shop,Venue Category_Motel,Venue Category_Moroccan Restaurant,Venue Category_Monument / Landmark,Venue Category_Molecular Gastronomy Restaurant,Venue Category_Modern European Restaurant,Venue Category_Miscellaneous Shop,Venue Category_Museum


### Cluster 1 and Cluster 4 are having most popular venues, while Cluster 2 , Cluster 3 and Cluster 5 being less popular